In [1]:
import sys
import matplotlib.pyplot as plt
import numpy as np
from astropy.io import fits
import pywt
from sklearn.decomposition import FastICA, PCA
from scipy.stats import pearsonr as pear

## Wavelet transform 

In [2]:
in_path_fg = "fits_files/"
in_path_cs = "Boxes/"

output_path = "Results/"
fname_all = "test_filename"

fname_foreground = "fg_all"
fname_cs = "test"

#1000 hours of noise
noise = fits.getdata(r'fits_files/noise_108.000MHz_210.300MHz_SKA_SKA_central_area_EOR0_0128_3.0_1000h_K.fits')
#100 hours
#noise = fits.getdata('fits_files/Noise_100.fits').T

indata_fg = fits.getdata(in_path_fg+fname_foreground+'.fits').T
indata_fg = indata_fg + noise

indata_cs = fits.getdata(in_path_cs+fname_cs+'.fits')
indata_cs = indata_cs/1000
sh = indata_cs.shape
print(sh)
print(indata_cs.shape)
no_sl = 342
#code for seperating by k-space
indata = indata_fg+indata_cs

(342, 128, 128)
(342, 128, 128)


In [3]:
# #############################################################################
# Generate sample data
#code for fastICA
all_cube_in = indata_fg+indata_cs

Dim = 128
nbrScale = 6

#observation
all_cube_in = (indata_fg+indata_cs).T
print(np.shape(all_cube_in))

# Convert to sparse domain
WT = np.zeros((Dim,Dim,nbrScale)) # Desired array where we have squashed together the fine/approximation bits
WTc_approx = np.zeros((Dim,Dim,nbrScale))
WTc_fine = np.zeros((3,Dim,Dim,nbrScale))

(128, 128, 342)


In [4]:
space_res = np.zeros((128,128,7,342))
for i in range(no_sl):
    coeffs = pywt.swt2(all_cube_in[:,:,i],'Haar',nbrScale,norm='True')
    
#Show the approximation and fine detail at each level
    for jj in range(0,nbrScale):
        WTc_approx[:,:,jj] = coeffs[jj][0]
        WTc_fine[:,:,:,jj] = coeffs[jj][1]
        space_res[:,:,0,i] = WTc_approx[:,:,0]
    
    # MERCILESSLY SQUASH THE ARRAYS
        WT[:,:,jj] = np.sum(WTc_fine[:,:,:,jj],axis=0)  # These are our wavelet coefficients
    

    space_res[:,:,1:7,i] = WT[:,:,0:6]

    #out = WTc_approx[:,:,0] + np.sum(WT[:,:,0:nbrScale],axis=2)

In [5]:
#compare k-space combination with the input data
#Wave transform
#plt.imshow(np.sum(space_res[:,:,0:7,0],axis=2))
#plt.colorbar()
#input data
#plt.figure()
#plt.imshow(all_cube_in[:,:,0])
#plt.colorbar()
#difference
#plt.figure()
#plt.imshow(all_cube_in[:,:,0] - np.sum(space[:,:,0:7,0],axis=2))
#plt.colorbar()

In [ ]:
res = np.zeros((342,128,128,7))
rec = np.zeros((342,128,128,7))

for i in range(7):
    indata = space_res[:,:,i,:].T
    sh = indata.shape
    print(sh)
    X = np.reshape(indata,(sh[0], sh[1]*sh[2]))
  
    ncomp = 3
        
    ica1 = FastICA(n_components=ncomp)
    model1 = []
    S = ica1.fit_transform(X.T)  # Reconstruct signals
    A = ica1.mixing_  # Get estimated mixing matrix

    model1 = np.reshape(np.matmul(A,S.T),(sh[0], sh[1],sh[2]))
    res[:,:,:,i] = indata-model1
    #rec[:,:,i,:] =  indata-res[:,:,i,:]


(342, 128, 128)
(342, 128, 128)


/Users/student2/opt/anaconda3/lib/python3.9/site-packages/sklearn/decomposition/_fastica.py:116: ConvergenceWarning: FastICA did not converge. Consider increasing tolerance or the maximum number of iterations.
  warnings.warn(


(342, 128, 128)


/Users/student2/opt/anaconda3/lib/python3.9/site-packages/sklearn/decomposition/_fastica.py:116: ConvergenceWarning: FastICA did not converge. Consider increasing tolerance or the maximum number of iterations.
  warnings.warn(


(342, 128, 128)


In [ ]:
#variance
res_2 = np.sum(res[:,:,:,0:8], axis = 3)
print(np.shape(res_2))
var_21 = np.zeros(no_sl)
var_res = np.zeros(no_sl)
recover = np.zeros(no_sl)
for i in range(no_sl):
    var_21[i] = np.var(indata_cs[i,:,:])
    var_res[i] = np.var(res_2[i,:,:])
    recover[i] = var_res[i] - np.var(noise[i])
    
       
#difference
diff = var_21 - var_res
diff2 = var_21 - recover

x = np.linspace(108,208.3,no_sl)


fig = plt.figure(figsize=(15,4))
ax = fig.add_subplot(1,2,1)
ax.title.set_text('Variance')
ax.set_xlabel('frequency (MHZ)')
im0 = ax.plot(x,var_21,'r',linewidth =3,label='21cmfast')
im1 = ax.plot(x,var_res,'g',label='residual')
im2 = ax.plot(x,recover, 'b.',markersize=1.5, label = 'recovered_signal')
ax.legend()

ax = fig.add_subplot(1,2,2)
im = ax.plot(x,diff, label =' 21cmsignal - residual')
im1 = ax.plot(x,diff2, label = '21cmsignal - recovered')
ax.title.set_text('Difference in variance')
ax.set_xlabel('frequency(MHZ)')
ax.legend()
#plt.show()
#fig.savefig('images/Variance_noise100.png')
#plt.close()

In [ ]:
resrem = np.sum(res[:,:,:,0:6],axis = 3) 
#turn into subplots
fig, axes = plt.subplots(3, 1, figsize=(6,12))

im1 =axes[0].imshow(resrem[100,:,:] - noise[100,:,:])
fig.colorbar(im1, ax=axes[0])
axes[0].axis('off')
axes[0].set_title('Taking the noise away from the residual')

im2 =axes[1].imshow(indata_cs[100,:,:])
fig.colorbar(im2, ax=axes[1])
axes[1].axis('off')
axes[1].set_title('21cmsignal')

im3 =axes[2].imshow(resrem[100])
fig.colorbar(im3, ax=axes[2])
axes[2].axis('off')
axes[2].set_title('Removing the last wavelet transform')

#plt.savefig('images/comparing WT with removing the noise')

## Correlate each WT of 21cm,noise and residual together

In [ ]:
import sys
import matplotlib.pyplot as plt
import numpy as np
from astropy.io import fits
import pywt
from sklearn.decomposition import FastICA, PCA
from scipy.stats import pearsonr as pear

In [ ]:
in_path_fg = "fits_files/"
in_path_cs = "Boxes/"

output_path = "Results/"
fname_all = "test_filename"

fname_foreground = "fg_all"
fname_cs = "test"

#1000 hours of noise
noise = fits.getdata(r'fits_files/noise_108.000MHz_210.300MHz_SKA_SKA_central_area_EOR0_0128_3.0_1000h_K.fits')
#100 hours
#noise = fits.getdata('fits_files/Noise_100.fits').T

indata_fg = fits.getdata(in_path_fg+fname_foreground+'.fits').T
indata_fg = indata_fg + noise

indata_cs = fits.getdata(in_path_cs+fname_cs+'.fits')
indata_cs = (indata_cs/1000)
sh = indata_cs.shape
print(sh)
print(indata_cs.shape)
no_sl = 342
#code for seperating by k-space
indata = indata_fg+indata_cs

In [ ]:
print(np.shape(noise))
plt.imshow(noise[10,:,:])

In [ ]:
# #############################################################################
# Generate sample data
#code for fastICA
all_cube_in = indata_fg+indata_cs

Dim = 128
nbrScale = 6

#observation
all_cube_in = (indata_fg+indata_cs).T
indata_cs = indata_cs.T
noise = noise.T

print(np.shape(all_cube_in))

# Convert to sparse domain
WT = np.zeros((Dim,Dim,nbrScale)) # Desired array where we have squashed together the fine/approximation bits
WTc_approx = np.zeros((Dim,Dim,nbrScale))
WTc_fine = np.zeros((3,Dim,Dim,nbrScale))
#for 21cmsignal
# Convert to sparse domain
WT_l = np.zeros((Dim,Dim,nbrScale)) # Desired array where we have squashed together the fine/approximation bits
WTc_approx_l = np.zeros((Dim,Dim,nbrScale))
WTc_fine_l = np.zeros((3,Dim,Dim,nbrScale))
#for noise
# Convert to sparse domain
WT_n = np.zeros((Dim,Dim,nbrScale)) # Desired array where we have squashed together the fine/approximation bits
WTc_approx_n = np.zeros((Dim,Dim,nbrScale))
WTc_fine_n = np.zeros((3,Dim,Dim,nbrScale))

In [ ]:
print(np.shape(noise))

In [ ]:
indata_slices = np.zeros((128,128,7,342))
light_slices = np.zeros((128,128,7,342))
noise_slices = np.zeros((128,128,7,342))


for i in range(no_sl):
    # indata coefficients
    coeffs = pywt.swt2(all_cube_in[:,:,i],'Haar',nbrScale,norm='True')
    # 21cmsignal
    coeffs_light = pywt.swt2(indata_cs[:,:,i],'Haar',nbrScale,norm='True')
    # noise
    coeffs_noise = pywt.swt2(noise[:,:,i],'Haar',nbrScale,norm='True')
    
#Show the approximation and fine detail at each level
    for jj in range(0,nbrScale):
        WTc_approx[:,:,jj] = coeffs[jj][0]; WTc_fine[:,:,:,jj] = coeffs[jj][1]
    # MERCILESSLY SQUASH THE ARRAYS
        WT[:,:,jj] = np.sum(WTc_fine[:,:,:,jj],axis=0)  # These are our wavelet coefficients
    
    #light
        WTc_approx_l[:,:,jj] = coeffs_light[jj][0]; WTc_fine_l[:,:,:,jj] = coeffs_light[jj][1]
    
    # MERCILESSLY SQUASH THE ARRAYS
        WT_l[:,:,jj] = np.sum(WTc_fine_l[:,:,:,jj],axis=0)  # These are our wavelet coefficient

    #Noise
        WTc_approx_n[:,:,jj] = coeffs_noise[jj][0]; WTc_fine_n[:,:,:,jj] = coeffs_noise[jj][1]
        noise_slices[:,:,0,i] = WTc_approx_n[:,:,0]
    
    # MERCILESSLY SQUASH THE ARRAYS
        WT_n[:,:,jj] = np.sum(WTc_fine_n[:,:,:,jj],axis=0)  # These are our wavelet coefficient
    
    #crude approximation
    indata_slices[:,:,0,i] = WTc_approx[:,:,0]
    light_slices[:,:,0,i] = WTc_approx_l[:,:,0]
    noise_slices[:,:,0,i] = WTc_approx_n[:,:,0]


     #finer approximation
    indata_slices[:,:,1:7,i] = WT[:,:,0:6]
    light_slices[:,:,1:7,i] = WT_l[:,:,0:6]
    noise_slices[:,:,1:7,i] = WT_n[:,:,0:6]

In [ ]:
res = np.zeros((342,128,128,7))
#rec = np.zeros((342,128,128,7))

for i in range(7):
    indata = indata_slices[:,:,i,:].T
    sh = indata.shape
    print(sh)
    X = np.reshape(indata,(sh[0], sh[1]*sh[2]))
  
    ncomp = 3
        
    ica1 = FastICA(n_components=ncomp)
    model1 = []
    S = ica1.fit_transform(X.T)  # Reconstruct signals
    A = ica1.mixing_  # Get estimated mixing matrix

    model1 = np.reshape(np.matmul(A,S.T),(sh[0], sh[1],sh[2]))
    res[:,:,:,i] = indata-model1
    #rec[:,:,i,:] =  indata-res[:,:,i,:]

In [ ]:
# so I have three WT: res, light_slices and, noise_slices
# make sure the shapes are the same for each of the three WT
light_slices = light_slices.T
noise_slices = noise_slices.T
print('Residual has the shape   =',np.shape(res))
print('21cmsignal has the shape =',np.shape(light_slices))
print('Noise has the shape      =',np.shape(noise_slices))

In [ ]:
# show a slice for each WT
#plt.figure()
#plt.imshow(res[100,:,:,4])
#plt.figure()
#plt.imshow(light_slices[100,4,:,:])
#plt.figure()
#plt.imshow(noise_slices[100,4,:,:])

In [ ]:
#show 3 columns and 8 rows of the WT slices

In [ ]:
# create figure and subplots
fig, axs = plt.subplots(4, 3, figsize=(8,6), gridspec_kw ={'hspace':0.25, 'wspace':0.0})

for col in range(4):
    #plot subplots
    #residual
    axs[col, 0].imshow(res[100,:,:,col])
    axs[col, 0].axis('off')
    #light
    axs[col, 1].imshow(light_slices[100,col,:,:])
    axs[col, 1].axis('off')
    #noise
    axs[col, 2].imshow(noise_slices[100,col,:,:])
    axs[col, 2].axis('off')
    
    axs[col, 0].text( -10, -10, f'Residual, W = {col+1}')
    
    axs[col, 1].text( -30, -10, f'Cosmological, W = {col+1}')

    axs[col, 2].text( 5, -10, f'Noise, W = {col+1}')

    plt.axis('off')


plt.tight_layout()
#plt.figure()
#plt.imshow(np.sum(noise_slices[10,0:8,:,:], axis = 0))
#plt.figure()
#plt.imshow(noise[:,:,10].T)
plt.savefig('Submit/comparing_WT_of_res_21cm_noise.png')

In [ ]:
# correlate the 21cmsignal and noise with the residual then, plot a graph of correlation.
corr_res_21 = np.zeros((7,342))
corr_res_noise = np.zeros((7,342))
for i in range(7):
    for m in range(342):
        residual = res[m,:,:,i]
        # you have to transpose the 21cmsignal and noise to get the correct orientation
        light = light_slices[m,i,:,:]
        noise = noise_slices[m,i,:,:]
        corr_res_21[i,m] = pear(residual.flatten(), light.flatten())[0]
        corr_res_noise[i,m] = pear(residual.flatten(), noise.flatten())[0]


In [ ]:
print(corr_res_21[:,0])
print(corr_res_noise[:,0])

In [ ]:
# Checking to see the data from 21cmsignal is the same as the combine WT
#plt.imshow(np.sum(light_slices[0,0:7,:,:],axis =0))
#plt.colorbar()
#plt.figure()
#plt.imshow(indata_cs[:,:,0].T)
#plt.colorbar()
#plt.figure()
#plt.imshow(indata_cs[:,:,0].T - np.sum(light_slices[0,0:7,:,:],axis =0))
#plt.colorbar()

### Important note: Using FastICA to get the residual results, Transposed the images. Need to make sure fast21cm and the noise is also transposed when  comparing.

In [ ]:
x = np.arange(1,8)
plt.figure()
plt.plot(x,corr_res_21[:,100],'.--', label = 'Cosmological signal')
plt.plot(x,corr_res_noise[:,100],'.--', label = 'Noise')
plt.grid(True, linestyle= '--', alpha = 0.5)
plt.xlabel('Wavelet number')
plt.ylabel('Correlation')
plt.title('Correlation with the residual Wavelets ', fontsize=12)
plt.legend()
#plt.savefig('images/correlation_with_residual_100slice.png')

In [ ]:
#average
mean_res_21 = np.zeros(7)
mean_res_noise = np.zeros(7)
for i in range(7):
    mean_res_21[i] = np.mean(corr_res_21[i,:])
    mean_res_noise[i] = np.mean(corr_res_noise[i,:])
print(np.shape(corr_res_21))

In [ ]:
x = np.arange(1,8)
plt.rcParams.update({'font.size' : 10})
plt.figure(figsize =(6,4))
plt.plot(x,mean_res_21,'.--', label = 'Cosmological signal')
plt.plot(x,mean_res_noise,'.--', label = 'Noise')
plt.grid(True, linestyle= '--', alpha = 0.5)
plt.xlabel('Wavelet number.')
plt.ylabel('Average correlation')
#plt.title('Average correlation with the residual wavelets', fontsize=12)
plt.legend()
plt.savefig('Submit/correlation_with_residual_WT_average.png')
print(mean_res_21)
print(mean_res_noise)

In [ ]:
test = np.sum(light_slices[150,0:7,:,:],axis=0)
actual = np.sum(res[150,:,:,0:7],axis=2)
plt.imshow(test)
plt.figure()
plt.imshow(actual)

In [ ]:
# Choosing the best slice by taking the average of the WT
all_res_slices = np.zeros(342)
all_noise_slices = np.zeros(342)

residual = np.sum(res[:,:,:,0:8], axis = 3)
light = np.sum(light_slices[:,0:8,:,:], axis = 1)
noise = np.sum(noise_slices[:,0:8,:,:], axis = 1)
#print shape size
print(np.shape(residual))
print(np.shape(light))
print(np.shape(noise))


for m in range(342):
        residua = residual[m,:,:]
        ligh = light[m,:,:]
        nois = noise[m,:,:]
        all_res_slices[m] = pear(residua.flatten(), ligh.flatten())[0]
        all_noise_slices[m] = pear(residua.flatten(), nois.flatten())[0]


# print max slice correlation
print('Max slice = ',np.argmax(all_res_slices))
print('Max slice corr value = ',all_res_slices[135])
#print min slice correlation
print('Min slices = ',np.argmin(all_res_slices))
print('Min value corr value = ',all_res_slices[0])

In [ ]:
best_slice = np.sum(res[135,:,:,0:8],axis = 2)
worst_slice = np.sum(res[0,:,:,0:8],axis = 2)
plt.rcParams.update({'font.size' : 10})



#enhance best slice image, bright spots brighter, dark spots darker
enhanced = np.zeros((128,128))
enhanced_worst = np.zeros((128,128))

for i in range(128):
    for m in range(128):
        best = best_slice[i,m]
        worst = worst_slice[i,m]
        if best > 0.0:
            enhanced[i,m] = best*10
        if worst > 0.0:
            enhanced_worst[i,m] = worst*10


            
# create figure and subplots
fig, axs = plt.subplots(2, 3, figsize=(8,6))

print(np.shape(indata_cs))

axs[0,0].imshow(best_slice); axs[0,0].axis('off'); axs[0,0].set_title('Best residual, corr = 0.70')

axs[0,1].imshow(enhanced); axs[0,1].axis('off'); axs[0,1].set_title('Enhanced residual')

axs[0,2].imshow(np.sum(light_slices[135,0:8,:,:], axis = 0)); axs[0,2].axis('off')
axs[0,2].set_title('Cosmological signal')


axs[1,0].imshow(worst_slice); axs[1,0].axis('off'); axs[1,0].set_title('Worst residual, corr = 0.19')

axs[1,1].imshow(enhanced_worst); axs[1,1].axis('off'); axs[1,1].set_title('Enhanced residual')

axs[1,2].imshow(np.sum(light_slices[0,0:8,:,:], axis = 0)); axs[1,2].axis('off')
axs[1,2].set_title('Cosmological signal')

plt.savefig('Submit/best_and_worst_residuals')


In [ ]:
freq = np.arange(108,210.6,0.3)
result_forward = np.zeros((342,7))
result_backwards = np.zeros((342,7))
for m in range(7):
    k = 7 - m
    for i in range(no_sl):
        light = np.sum(light_slices[i,0:7,:,:],axis=0)
        residual = np.sum(res[i,:,:,m:7],axis=2)
        result_forward[i,m] = pear(light.flatten(),residual.flatten())[0]
        resi = np.sum(res[i,:,:,0:k],axis=2)
        result_backwards[i,m] = pear(light.flatten(),resi.flatten())[0]


    

In [ ]:
fig, axes = plt.subplots(1,2, figsize =(14,5))
for m in range(7):
    axes[0].plot(freq[::4],result_forward[::4,m], label = "No. of slices removed = "+str(m))
    axes[0].legend(fontsize = 7)
    axes[0].grid()
    axes[0].set_xlabel('Frequency (Hz)')
    axes[0].set_ylabel('Correlation')
    axes[0].set_ylim(0.1,0.8)
    axes[0].set_title('Correlation with 21cmsignal, remove from largest k-space from WT')


    axes[1].plot(freq[::4], result_backwards[::4,m], label = "No. of slices removed = "+str(m))
    axes[1].legend()
    axes[1].grid()
    axes[1].set_xlabel('Frequency (Hz)')
    axes[1].set_ylabel('Correlation')
    axes[1].set_ylim(-0.12,0.85)
    axes[1].legend(fontsize = 7)
    axes[1].set_title('Correlation with 21cmsignal, remove from smallest k-space from WT')
plt.tight_layout()
#plt.savefig('images/corr 21cmsignal WT k-space')

In [ ]:
plt.figure(figsize=(8,5))
plt.rcParams.update({'font.size' : 10})

for m in range(7):
    plt.plot(freq[::4],result_forward[::4,m], label = "No. of slices removed = "+str(m))
    plt.legend(fontsize = 7)
plt.grid(True, linestyle= '--', alpha = 0.5)
plt.xlabel('Frequency (Hz)')
plt.ylabel('Correlation')
plt.ylim(0.1,0.8)
plt.tight_layout()
plt.savefig('Submit/Residual_corr_21_remove_smallest_k-space.png')

In [ ]:
plt.figure(figsize=(8,5))
plt.rcParams.update({'font.size' : 8})

for m in range(7):
    plt.plot(freq[::4],result_backwards[::4,m], label = "Wavelets removed = "+str(m))
    plt.legend()
plt.grid(True, linestyle= '--', alpha = 0.5)
plt.xlabel('Frequency (Hz)')
plt.ylabel('Correlation')
plt.ylim(-0.12,0.85)
plt.tight_layout()
plt.savefig('Submit/Residual_corr_21_remove_largest_k-space.png')